In [1]:
!git clone https://github.com/huggingface/transformers

fatal: destination path 'transformers' already exists and is not an empty directory.


In [2]:
song=[]
!mkdir content

mkdir: cannot create directory ‘content’: File exists


In [3]:
import os
#os.chdir("transformers")
os.chdir("./transformers/examples/tensorflow/language-modeling")

In [4]:
!pip install -r  requirements.txt
!pip install pyarrow --upgrade

In [5]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-i__wdhhy
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-i__wdhhy
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [6]:
%cd /content

/content


In [7]:

"""
Refer: https://towardsdatascience.com/natural-language-generation-part-2-gpt-2-and-huggingface-f3acb35bc86a
"""
from sklearn.model_selection import train_test_split

with open('song.txt', 'r') as data:
  dataset = ["<|title|>" + x.strip() for x in data.readlines()]

train, eval = train_test_split(dataset, train_size=.9, random_state=2020)
print("training size:" + str(len(train)))
print("Evaluation size: " + str(len(eval)))

with open('train.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(train))

with open('eval.txt', 'w') as file_handle:
  file_handle.write("<|endoftext|>".join(eval))

training size:65885
Evaluation size: 7321


In [8]:
%cd /content/transformers/examples/tensorflow/language-modeling

/content/transformers/examples/tensorflow/language-modeling


In [9]:
%cp /content/train_tmp.txt /content/transformers/examples/tensorflow/language-modeling/
%cp /content/eval_tmp.txt /content/transformers/examples/tensorflow/language-modeling/

In [10]:
"""
https://towardsdatascience.com/natural-language-generation-part-2-gpt-2-and-huggingface-f3acb35bc86a
"""
!python run_clm.py \
--model_type distilgpt2 \
--model_name_or_path distilgpt2 \
--train_file "train.txt" \
--do_train \
--validation_file "eval.txt" \
--do_eval \
--per_gpu_train_batch_size 1 \
--save_steps -1 \
--num_train_epochs 15 \
--fp16 \
--output_dir="/content/mymodel"

Using custom data configuration default-391039a85029350f
Extracting data files: 100% 2/2 [00:00<00:00, 1033.97it/s]
Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-391039a85029350f/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.
100% 2/2 [00:00<00:00, 924.47it/s]
loading configuration file https://huggingface.co/distilgpt2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/f985248d2791fcff97732e4ee263617adec1edb5429a2b8421734c6d14e39bee.422318838d1ec4e061efb4ea29671cb2a044e244dc69229682bebd7cacc81631
Model config GPT2Config {
  "_name_or_path": "distilgpt2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "

In [11]:
from transformers import TFGPT2LMHeadModel
from transformers import GPT2Tokenizer

model = TFGPT2LMHeadModel.from_pretrained("/content/mymodel/", from_pt=False)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at /content/mymodel/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [19]:
input_ids = tokenizer.encode("I love deep learning", return_tensors='tf')
generated_text_samples = model.generate(
    input_ids, 
    max_length=10000,
    min_length = 500,
    num_return_sequences=20,
    no_repeat_ngram_size=2,
    repetition_penalty=2.5,
    top_p=0.92,
    temperature=.9,
    do_sample=True,
    top_k=125,
    early_stopping=True
)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


In [27]:
index=[3,5,6,7,11,13,14,15,17,18,20]
#Print output for each sequence generated above
import re
itr=0
for i, b in enumerate(generated_text_samples):
  if(i in index ):
      itr+=1
      song = tokenizer.decode(b, skip_special_tokens=True)
      
      print("=================================Song "+str(itr)+"===============================")
     
      #re.split(r"[^a-zA-Z0-9\s]", song)
     
      x=song.split('.')
      for s in x:
        if(s.strip()!=''):
          print(s)

      print()

=================================Song 1===============================
I love deep learning, and if I didn't know what to do then my mind was on the dance floor
 And so it's no surprise that you just left me here hoping your heart wouldn’t cry or cry inside of nobody when all around dies at our hands; oh yes! Well now this is not a crime against humanity but rather an unfortunate twist in nature? The way we walk those streets leads us deeper than they would let you into their lives
" He said "In every homey housewife working for nine days" We will lay down roots with each other right underneath his broken nose--we'll teach them how can one overcome another without breaking up again!" Oh please please don`T make him strong enough yet he won'n be able prove himself today If only somebody knew who else had heard all about
 When there were lies in every American dream they'd never turn aside until facts proved them wrongs [Chorus] So tell 'em why are these things happening? Do unto others 